Other languages with
gradual type systems are Microsoft’s TypeScript, Dart (the language of the
Flutter SDK, created by Google), and Hack (a dialect of PHP supported by
Facebook’s HHVM virtual machine).

Types are defined by supported operations

In [1]:
from collections import abc
def double(x: abc.Sequence):
    return x * 2


In [3]:
def tokenize(text: str) -> list[str]:
    return text.upper().split()


tokenize('word one two')


['WORD', 'ONE', 'TWO']

In [1]:
from geolib import geohash as gh

ardabil = 28.3, 47.2
gh(ardabil)

TypeError: 'module' object is not callable

In [2]:
PRECISION = 9
def geohash(lat_lon: tuple[float, float]) -> str:
    return gh.encode(*lat_lon, PRECISION)

In [5]:
geohash((28.5,26.2))

'st4dhhq28'

In [6]:
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

def geohash2(lat_lon= Coordinate) -> str:
    return gh.encode(*lat_lon, PRECISION)

In [7]:
def display(lat_lon: tuple[float, float]) -> str:
    lat, lon = lat_lon
    ns = 'N' if lat >= 0 else 'S'
    ew = 'E' if lon >= 0 else 'W'
    return f'{abs(lat):0.1f}°{ns}, {abs(lon) :0.1f}°{ew}'

In [8]:
display((34.4, 33.3))

'34.4°N, 33.3°E'

In [11]:
import columnize
from collections.abc import Sequence

def columnize1(
    sequence: Sequence[str],
     num_columns: int = 0
     ) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    num_rows, remainder = divmod(len(sequence), num_columns)
    num_rows += bool(remainder)
    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]


animals = 'drake fawn heron ibex koala lynx tahr xerus yak zapus'.split()
table = columnize.columnize(animals, displaywidth=16, colsep=' | ' )
table2 = columnize1(animals)

In [12]:
table2

[('drake', 'koala', 'yak'),
 ('fawn', 'lynx', 'zapus'),
 ('heron', 'tahr'),
 ('ibex', 'xerus')]

In [17]:
for row in table2:
    print('__________________________________')
    print(''.join(f'{word:10}|' for word in row))


__________________________________
drake     |koala     |yak       |
__________________________________
fawn      |lynx      |zapus     |
__________________________________
heron     |tahr      |
__________________________________
ibex      |xerus     |


### Generic mappings

Generic mapping types are annotated as MappingType[KeyType,
ValueType]

Example 8-16 defines sample, a function that takes two arguments: a
Sequence of elements of type T, and an int. It returns a list of
elements of the same type T, picked at random from the first argument.

In [2]:
from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

T = TypeVar('T')

In [3]:
def sample(population: Sequence[T], size: int) -> list[T]:
    if size < 1:
        raise ValueError('size must be >= 1')
    result = list(population)
    shuffle(result)
    return result[:size]


Without using a TypeVar, mode could have this signature:
Example 8-17. mode_float.py: mode that operates on float and
subtypes.

In [5]:
from collections import Counter
from collections.abc import Iterable, Hashable
from decimal import Decimal
from fractions import Fraction
from typing import TypeVar

NumberT = TypeVar('NumberT', float, Decimal, Fraction)

def mode(data: Iterable[NumberT]) -> NumberT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]


In [6]:
mode(["1", "1", "2", "3", "3", "3", "3", "4"])

'3'

Example 8-18. mode_hashable.py: same as Example 8-17, with a more
flexible signature.

In [7]:
HashableT = TypeVar('HashableT', bound=Hashable)

def mode(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

In [8]:
mode(["1", "1", "2", "3", "3", "3", "3", "4"])

'3'

A parameterized generic top would look like this:
Example 8-19. top function with an undefined T type parameter.

In [11]:
T = TypeVar('AnyStr', bytes, str)

def top(series: Iterable[T], length: int) -> list[T]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

In [12]:
l = 'mango pear apple kiwi banana'.split()
top(l, 3)

['pear', 'mango', 'kiwi']

Example 8-20. comparable.py: definition of a SupportsLessThan
Protocol type:

In [13]:
from typing import Protocol, Any

class SupportLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...
    

Example 8-21. top.py: definition of the top function using a TypeVar
with bound=SupportsLessThan:

In [ ]:
LT = TypeVar('LT', bound=SupportLessThan)

def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]
